In [1]:
import pandas as pd
import numpy as np

# 假設資料來源
df = pd.read_csv("all_dataset.csv")

# Step 1. 移除 type 為 "接不到" 的列
df = df[df['type'] != '接不到']

# 建立遮罩處理 getpoint_player
mask_not_last_in_rally = df.duplicated(subset=['rally_id'], keep='last')
df.loc[mask_not_last_in_rally, 'getpoint_player'] = np.nan
print("✅ 'getpoint_player' 處理完畢。")

# Step 2. 若 lose_reason 或 win_reason 包含「對手」則改為 NaN
for col in ['lose_reason', 'win_reason']:
    df[col] = df[col].apply(lambda x: np.nan if isinstance(x, str) and "對手" in x else x)

# Step 3. 每場比賽的每個 set 最後插入一筆新資料，並新增 opponent 欄位
# ----------------------------------------------------

dfs_to_concat = []

# 分組：每場比賽 + 該比賽的每個 set
for (match_id, set_id), group in df.groupby(['match_id', 'set'], sort=False):
    # 為了避免 SettingWithCopyWarning，先對 group 進行 copy
    group = group.copy()

    # --- [新增功能] 處理 opponent 欄位 ---
    # 1. 找出這個 Set 裡面的所有球員 (理論上應該只有 2 位)
    players = group['player'].unique()
    
    # 確保只有兩位球員才進行對應 (避免資料錯誤導致 crash)
    if len(players) == 2:
        p1, p2 = players[0], players[1]
        # 建立互換字典：如果是 p1 對手就是 p2；如果是 p2 對手就是 p1
        opponent_map = {p1: p2, p2: p1}
        # 使用 map 快速填入 opponent
        group['opponent'] = group['player'].map(opponent_map)
    else:
        # 如果資料異常(例如只有1個球員或是多於2個)，填入 NaN 或保留原樣
        group['opponent'] = np.nan
    # ----------------------------------

    # 1. 先把該組原本的資料 (現在已經包含 opponent 欄位) 加入 list
    dfs_to_concat.append(group)

    # 2. 取該 Set 的最後一筆資料並複製
    # 注意：因為上面已經處理了 opponent，這裡 copy 出來的 last_row 也會自動帶有 opponent 欄位
    last_row = group.iloc[-1].copy()
    
    winner = last_row['getpoint_player']
    
    if winner == 'CHOU Tien Chen':
        # 周天成得分 -> opponent_score (周天成自己的分數) + 1
        last_row['opponent_score'] += 1
    else:
        # 對手得分 -> player_score (對手的分數) + 1
        last_row['player_score'] += 1

    # 將這筆新增的資料轉為 DataFrame 並加入 list
    dfs_to_concat.append(pd.DataFrame([last_row]))

# 4. 最後一次合併所有資料
new_df = pd.concat(dfs_to_concat, ignore_index=True)

# 輸出結果
new_df.to_csv("processed_new_2.csv", index=False)
print("✅ 已完成：新增 opponent 欄位，並插入更新比分的新資料。")

✅ 'getpoint_player' 處理完畢。
✅ 已完成：新增 opponent 欄位，並插入更新比分的新資料。


In [2]:
import pandas as pd
import sqlite3

# 讀取 CSV 檔
csv_file = "processed_new_2.csv"
df = pd.read_csv(csv_file)

# 建立 SQLite 資料庫（若不存在會自動建立）
db_file = "processed_new_2.db"
conn = sqlite3.connect(db_file)

# 將 DataFrame 寫入 SQLite，表格名稱可自訂
table_name = "match_data"
df.to_sql(table_name, conn, if_exists="replace", index=False)

# 確認寫入完成
print(f"✅ 已將 {csv_file} 匯入 SQLite 資料庫 {db_file}，表格名稱：{table_name}")

# （可選）查看前 5 筆資料確認
print(pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5;", conn))

# 關閉連線
conn.close()


✅ 已將 processed_new_2.csv 匯入 SQLite 資料庫 processed_new_2.db，表格名稱：match_data
   rally  ball_round      time  frame_num  score_status  player_score  \
0      1           1   0:07:39    11496.0           0.0           0.0   
1      1           2  00:07:43    11582.0           0.0           0.0   
2      2           1   0:07:55    11881.0           1.0           1.0   
3      2           2   0:07:56    11900.0           1.0           1.0   
4      2           3   0:07:56    11921.0           1.0           1.0   

   opponent_score          player  server type  ...  moving_x  moving_y  \
0             0.0    Kento MOMOTA     1.0  發短球  ...  0.012041  0.262869   
1             0.0  CHOU Tien Chen     3.0   長球  ... -0.073337  0.594564   
2             0.0    Kento MOMOTA     1.0  發短球  ...  0.011994  0.275664   
3             0.0  CHOU Tien Chen     2.0  推撲球  ... -0.058087  0.142593   
4             0.0    Kento MOMOTA     2.0   殺球  ...  0.158490 -0.000140   

   landing_court_number  ball_distan